# NEWS2PROFIT - Data Exploration and Analysis

This notebook demonstrates how to use the NEWS2PROFIT project for stock market prediction using machine learning and financial news sentiment analysis.

## Project Overview
- **Objective**: Predict NSE stock movement (UP/DOWN/NEUTRAL) using historical data and news sentiment
- **Data Sources**: yfinance for stock data, NewsAPI/Twitter/Reddit for news
- **ML Models**: Logistic Regression, XGBoost, LSTM
- **Features**: Technical indicators (SMA, EMA, RSI, MACD) + sentiment scores

## Notebook Structure
1. Import libraries and setup
2. Data collection from multiple sources
3. Data preprocessing and feature engineering
4. Sentiment analysis of news data
5. Model training and evaluation
6. Visualization and insights

In [44]:
# Import Required Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Import NEWS2PROFIT modules with error handling
import sys
import os
sys.path.append(os.path.join(os.getcwd(), '..'))

try:
    from config.config import *
    print(" Config imported successfully!")
except ImportError as e:
    print(f"Config import error: {e}")

try:
    from src.data_collection import StockDataCollector, NewsDataCollector
    print(" Data collection modules imported successfully!")
except ImportError as e:
    print(f" Data collection import error: {e}")

try:
    from src.preprocessing import StockDataPreprocessor, NewsDataPreprocessor, DataIntegrator
    print("Preprocessing modules imported successfully!")
except ImportError as e:
    print(f"Preprocessing import error: {e}")

try:
    from src.sentiment import SentimentAnalyzer
    print("Sentiment analysis module imported successfully!")
except ImportError as e:
    print(f" Sentiment analysis import error: {e}")
    print("Try installing missing dependencies: pip install textblob vaderSentiment")

try:
    from src.model import StockPredictor, LogisticRegressionModel
    print("Model modules imported successfully!")
except ImportError as e:
    print(f" Model import error: {e}")

# Set plotting style
try:
    plt.style.use('seaborn-v0_8')
    sns.set_palette("husl")
    print(" Plotting style set successfully!")
except Exception as e:
    print(f" Plotting style warning: {e}")
    # Fallback to default style
    sns.set_palette("deep")

print(" All available libraries imported!")
try:
    print(f" Selected stocks: {NSE_STOCKS[:5]}...")  # Show first 5 stocks
except NameError:
    print(" Config not loaded - will use default stock selection")

 Config imported successfully!
 Data collection modules imported successfully!
Preprocessing modules imported successfully!
Sentiment analysis module imported successfully!
Model modules imported successfully!
 Plotting style set successfully!
 All available libraries imported!
 Selected stocks: ['RELIANCE.NS', 'TCS.NS', 'HDFCBANK.NS', 'INFY.NS', 'ICICIBANK.NS']...


In [45]:
# Reload updated project modules to ensure latest code is used
import importlib
import src.data_collection as dc
import src.preprocessing as pp
import src.sentiment as se
import src.model as md

importlib.reload(dc)
importlib.reload(pp)
importlib.reload(se)
importlib.reload(md)

from src.data_collection import StockDataCollector, NewsDataCollector
from src.preprocessing import StockDataPreprocessor, NewsDataPreprocessor, DataIntegrator
from src.sentiment import SentimentAnalyzer
from src.model import StockPredictor, LogisticRegressionModel

print("Modules reloaded successfully.")

Modules reloaded successfully.


## 1. Data Collection

First, let's collect stock data from yfinance and news data from various sources.

In [46]:
# Collect Stock Data (with local CSV fallback)
print("Collecting stock data...")

try:
    import os
    import pandas as pd

    # Initialize stock data collector
    stock_collector = StockDataCollector()
    print("StockDataCollector initialized successfully!")

    # Focus on 5 specific NSE stocks for analysis
    selected_stocks = ['RELIANCE.NS', 'TCS.NS', 'INFY.NS', 'HDFCBANK.NS', 'ICICIBANK.NS']

    # Fetch 6 months of historical data (collector has internal fallbacks)
    stock_data_dict = stock_collector.fetch_stock_data(selected_stocks, period="6mo")

    # Combine all stock data from collector
    all_stock_data = []
    for symbol, data in (stock_data_dict or {}).items():
        if isinstance(data, pd.DataFrame) and not data.empty:
            all_stock_data.append(data)

    def _symbol_from_filename(fname: str) -> str:
        base = os.path.basename(fname)
        if base.endswith('_historical.csv'):
            base = base[:-len('_historical.csv')]
        # Map *_NS to *.NS
        if base.endswith('_NS'):
            return base.replace('_NS', '.NS')
        return base

    # If online fetch failed, attempt local CSV fallback from project root
    if not all_stock_data:
        print("No Yahoo data returned; attempting local CSV fallback from data/raw ...")
        project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
        raw_dir = os.path.join(project_root, 'data', 'raw')
        if os.path.exists(raw_dir):
            files = [f for f in os.listdir(raw_dir) if f.endswith('_historical.csv')]
            if files:
                fallback_frames = []
                for f in files:
                    fpath = os.path.join(raw_dir, f)
                    try:
                        df = pd.read_csv(fpath)
                        if df is None or df.empty:
                            continue
                        # Normalize basic columns
                        cols = {c: c.lower().strip().replace(' ', '_') for c in df.columns}
                        df = df.rename(columns=cols)
                        if 'date' not in df.columns and 'datetime' in df.columns:
                            df = df.rename(columns={'datetime': 'date'})
                        if 'symbol' not in df.columns or df['symbol'].isna().all():
                            df['symbol'] = _symbol_from_filename(f)
                        if 'close' not in df.columns and 'adj_close' in df.columns:
                            df['close'] = df['adj_close']
                        if 'volume' not in df.columns:
                            df['volume'] = pd.NA
                        if 'date' in df.columns:
                            df['date'] = pd.to_datetime(df['date'], errors='coerce')
                            df = df.dropna(subset=['date'])
                            df['date'] = df['date'].dt.date
                        # Keep standard columns where possible
                        keep = [c for c in ['date', 'open', 'high', 'low', 'close', 'volume', 'symbol'] if c in df.columns]
                        if keep:
                            fallback_frames.append(df[keep].copy())
                    except Exception as e:
                        print(f"Error reading {fpath}: {e}")
                if fallback_frames:
                    combined_stock_data = pd.concat(fallback_frames, ignore_index=True)
                    combined_stock_data = combined_stock_data.sort_values(['symbol', 'date'])
                    print(f"Loaded fallback data: {combined_stock_data.shape}")
                    print(f"Date range: {combined_stock_data['date'].min()} to {combined_stock_data['date'].max()}")
                    print(f"Stocks: {sorted(combined_stock_data['symbol'].dropna().unique())}")
                else:
                    print("Found CSVs but none contained valid data")
                    combined_stock_data = pd.DataFrame()
            else:
                print("No *_historical.csv files found in data/raw")
                combined_stock_data = pd.DataFrame()
        else:
            print(f"data/raw directory not found at: {raw_dir}")
            combined_stock_data = pd.DataFrame()
    else:
        # We have online or module-fallback data
        combined_stock_data = pd.concat(all_stock_data, ignore_index=True)
        if 'date' in combined_stock_data.columns:
            combined_stock_data = combined_stock_data.sort_values(['symbol', 'date'])
        print(f"Collected stock data: {combined_stock_data.shape}")
        print(f"Date range: {combined_stock_data['date'].min()} to {combined_stock_data['date'].max()}")
        print(f"Stocks: {sorted(combined_stock_data['symbol'].dropna().unique())}")
        # Save only if the collector has populated data internally
        stock_collector.save_stock_data()
        print("Stock data saved to CSV files")

except NameError as e:
    print(f"Import error: {e}")
    print("Please run cell 2 (imports) and the reload cell before this.")
    import pandas as pd
    combined_stock_data = pd.DataFrame()
    print("Created empty DataFrame as fallback")

except Exception as e:
    print(f"Error collecting stock data: {e}")
    import pandas as pd
    combined_stock_data = pd.DataFrame()
    print("Created empty DataFrame as fallback")

INFO:src.data_collection:Fetching stock data for 5 symbols


StockDataCollector initialized successfully!


RELIANCE.NS: No data found for this date range, symbol may be delisted
[*********************100%***********************]  1 of 1 completed

INFO:src.data_collection:Loaded 495 fallback records for RELIANCE.NS
INFO:src.data_collection:Loaded 495 fallback records for RELIANCE.NS




1 Failed download:
- RELIANCE.NS: No data found for this date range, symbol may be delisted


...

INFO:src.data_collection:Loaded 495 fallback records for HDFCBANK.NS
INFO:src.data_collection:Loaded 495 fallback records for HDFCBANK.NS




1 Failed download:
- HDFCBANK.NS: No data found for this date range, symbol may be delisted


ICICIBANK.NS: No data found for this date range, symbol may be delisted
[*********************100%***********************]  1 of 1 completed

INFO:src.data_collection:Loaded 495 fallback records for ICICIBANK.NS
INFO:src.data_collection:Loaded 495 fallback records for ICICIBANK.NS
INFO:src.data_collection:Saved RELIANCE.NS data to d:\NEWS2PROFIT\data\raw\RELIANCE_NS_historical.csv
INFO:src.data_collection:Saved TCS.NS data to d:\NEWS2PROFIT\data\raw\TCS_NS_historical.csv
INFO:src.data_collection:Saved INFY.NS data to d:\NEWS2PROFIT\data\raw\INFY_NS_historical.csv
INFO:src.data_collection:Saved RELIANCE.NS data to d:\NEWS2PROFIT\data\raw\RELIANCE_NS_historical.csv
INFO:src.data_collection:Saved TCS.NS data to d:\NEWS2PROFIT\data\raw\TCS_NS_historical.csv
INFO:src.data_collection:Saved INFY.NS data to d:\NEWS2PROFIT\data\raw\INFY_NS_historical.csv
INFO:src.data_collection:Saved HDFCBANK.NS data to d:\NEWS2PROFIT\data\raw\HDFCBANK_NS_historical.csv
INFO:src.data_collection:Saved ICICIBANK.NS data to d:\NEWS2PROFIT\data\raw\ICICIBANK_NS_historical.csv
INFO:src.data_collection:Saved HDFCBANK.NS data to d:\NEWS2PROFIT\data\raw\HDF



1 Failed download:
- ICICIBANK.NS: No data found for this date range, symbol may be delisted
Collected stock data: (2475, 7)
Date range: 2023-11-13 to 2025-11-13
Stocks: ['HDFCBANK.NS', 'ICICIBANK.NS', 'INFY.NS', 'RELIANCE.NS', 'TCS.NS']
Stock data saved to CSV files


In [47]:
# Collect News Data for 5 Specific Stocks
print("Collecting news data for RELIANCE, TCS, INFY, HDFCBANK, ICICIBANK...")

# Check if required modules are available
try:
    # Initialize news data collector
    news_collector = NewsDataCollector()
    print("NewsDataCollector initialized successfully!")
    
    # Fetch news data from the last 30 days for specific stocks + business headlines
    # This will automatically save data to separate CSV files for each source
    news_data = news_collector.fetch_all_news_data(days_back=30, save_to_csv=True)
    
    if not news_data.empty:
        print(f"Collected news data: {news_data.shape}")
        print(f"News sources: {news_data['source'].value_counts().to_dict()}")
        print(f"Date range: {news_data['date'].min()} to {news_data['date'].max()}")
        print(f"Keywords covered: {news_data['keyword'].unique()[:10]}")  # Show first 10 keywords
        print("News data saved to separate CSV files by source (NewsAPI, Twitter, Reddit)")
    else:
        print("No news data collected. Check API keys in .env file.")
        
except NameError as e:
    print(f"Import error: {e}")
    print("Please run cell 2 first to import all required modules")
    # Create empty DataFrame as fallback
    import pandas as pd
    news_data = pd.DataFrame()
    print("Created empty DataFrame as fallback")
    
except Exception as e:
    print(f"News collection failed: {str(e)}")
    print("Tip: Make sure API keys are properly set in .env file")
    # Create empty DataFrame for demo purposes
    import pandas as pd
    news_data = pd.DataFrame()
    
# Display what files were created
import os
if os.path.exists('data/raw'):
    print("\nCSV files created:")
    for file in os.listdir('data/raw'):
        if file.endswith('.csv'):
            print(f" {file}")
else:
    print("\n No data directory found - will be created when data is collected")

INFO:src.data_collection:Reddit API initialized successfully
INFO:src.data_collection:Fetching news from all sources for 5 specific stocks...
INFO:src.data_collection:Fetching NewsAPI data...
INFO:src.data_collection:Fetching stock news from SerpApi Google News Light...
INFO:src.data_collection:Fetching news for RELIANCE...
INFO:src.data_collection:Fetching news from all sources for 5 specific stocks...
INFO:src.data_collection:Fetching NewsAPI data...
INFO:src.data_collection:Fetching stock news from SerpApi Google News Light...
INFO:src.data_collection:Fetching news for RELIANCE...


NewsDataCollector initialized successfully!


INFO:src.data_collection:Fetched 10 articles for RELIANCE
INFO:src.data_collection:Fetching news for TCS...
INFO:src.data_collection:Fetching news for TCS...
INFO:src.data_collection:Fetched 10 articles for TCS
INFO:src.data_collection:Fetched 10 articles for TCS
INFO:src.data_collection:Fetching news for INFY...
INFO:src.data_collection:Fetching news for INFY...
INFO:src.data_collection:Fetched 10 articles for INFY
INFO:src.data_collection:Fetched 10 articles for INFY
INFO:src.data_collection:Fetching news for HDFCBANK...
INFO:src.data_collection:Fetching news for HDFCBANK...
INFO:src.data_collection:Fetched 10 articles for HDFCBANK
INFO:src.data_collection:Fetched 10 articles for HDFCBANK
INFO:src.data_collection:Fetching news for ICICIBANK...
INFO:src.data_collection:Fetching news for ICICIBANK...
INFO:src.data_collection:Fetched 10 articles for ICICIBANK
INFO:src.data_collection:Fetched 10 articles for ICICIBANK
INFO:src.data_collection:Total articles fetched: 50
INFO:src.data_coll

Collected news data: (360, 10)
News sources: {'reddit': 313, 'serpapi_google_news': 47}
Date range: 2025-10-17 to 2025-11-13
Keywords covered: ['RELIANCE' 'TCS' 'INFY' 'HDFCBANK' 'ICICIBANK' 'IndiaInvestments'
 'indianstreetbets' 'StockMarket' 'investing']
News data saved to separate CSV files by source (NewsAPI, Twitter, Reddit)

CSV files created:
 combined_data_20251113_220348.csv
 combined_data_20251113_221113.csv
 combined_data_20251113_221755.csv
 newsapi_data_20251113_220335.csv
 newsapi_data_20251113_221059.csv
 newsapi_data_20251113_221740.csv
 reddit_data_20251113_220348.csv
 reddit_data_20251113_221113.csv
 reddit_data_20251113_221754.csv


## 2. Data Exploration and Visualization

Let's explore the collected stock data with some visualizations.

In [48]:
# Display basic information about stock data
# Check if combined_stock_data exists and handle gracefully
try:
    if 'combined_stock_data' in locals() and not combined_stock_data.empty:
        print("Stock Data Overview")
        print("=" * 40)
        print(f"Shape: {combined_stock_data.shape}")
        print(f"Columns: {list(combined_stock_data.columns)}")
        print(f"Memory usage: {combined_stock_data.memory_usage().sum() / 1024:.2f} KB")
        
        # Display first few rows
        print("\n Sample Data:")
        display(combined_stock_data.head())
        
        # Basic statistics
        print("\n Basic Statistics:")
        display(combined_stock_data.describe())
    else:
        print(" No stock data available to display")
        print("\n To fix this issue:")
        print("1. Run Cell 4 (Data Collection) first to collect stock data")
        print("2. Make sure the stock data collection completed successfully")
        print("3. Then run this cell again")
        
        # Check if we can load existing data from files
        import os
        import pandas as pd
        
        raw_data_path = "data/raw"
        if os.path.exists(raw_data_path):
            csv_files = [f for f in os.listdir(raw_data_path) if f.endswith('_historical.csv')]
            if csv_files:
                print(f"\n Found {len(csv_files)} existing stock data files:")
                for file in csv_files:
                    print(f"   - {file}")
                print("\nYou can load existing data by running the data collection cell.")
            else:
                print("\n No existing stock data files found in data/raw/")
        else:
            print(f"\n Data directory '{raw_data_path}' not found")

except NameError:
    print(" combined_stock_data is not defined")
    print("\n To fix this:")
    print("1. First run Cell 2 (Import Required Libraries)")
    print("2. Then run Cell 4 (Collect Stock Data)")
    print("3. Finally run this cell again")
    print("\n Make sure to run cells in order for the best experience!")

except Exception as e:
    print(f"Unexpected error: {e}")
    print("Please check if all previous cells ran successfully.")

Stock Data Overview
Shape: (2475, 7)
Columns: ['date', 'open', 'high', 'low', 'close', 'volume', 'symbol']
Memory usage: 154.69 KB

 Sample Data:


,date,open,high,low,close,volume,symbol
1485,2023-11-13,726.183786,728.057514,723.166358,724.577759,20605914,HDFCBANK.NS
1486,2023-11-15,732.024014,735.357800,730.296316,732.170044,41245460,HDFCBANK.NS
1487,2023-11-16,734.749421,739.275622,731.853665,734.092407,29713838,HDFCBANK.NS
1488,2023-11-17,725.161739,733.678739,724.675053,732.510681,23701874,HDFCBANK.NS
1489,2023-11-20,736.355499,736.355499,730.782971,732.559326,19331452,HDFCBANK.NS



 Basic Statistics:


,open,high,low,close,volume
count,2475.000000,2475.000000,2475.000000,2475.000000,2.475000e+03
mean,1740.656942,1756.246584,1725.735160,1740.644660,1.347881e+07
std,1010.307521,1019.939513,1000.996412,1009.726047,1.843691e+07
min,669.679629,677.223258,663.620415,673.597473,0.000000e+00
25%,1102.085478,1112.391494,1093.205987,1102.921692,4.757562e+06
50%,1408.000000,1418.852051,1397.715108,1407.943604,9.269259e+06
75%,1772.531271,1793.617368,1754.137447,1771.790039,1.616354e+07
max,4411.415210,4427.080111,4349.717095,4389.964844,4.453421e+08


In [49]:
# Stock Price and Volume Visualization (robust version)
# - Imports required libraries (Plotly, pandas, os)
# - Uses existing `combined_stock_data` if available
# - Falls back to loading CSVs from ../data/raw/*_historical.csv (project root)
# - Normalizes columns to: date, close, volume, symbol
# - Plots price (line) and volume (bars) per symbol

try:
    import os
    import pandas as pd
    import numpy as np
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots

    def _normalize_df(df: pd.DataFrame) -> pd.DataFrame:
        # Lowercase columns and standardize common names
        colmap = {c: c.lower().strip().replace(' ', '_') for c in df.columns}
        df = df.rename(columns=colmap)
        # Ensure expected columns exist if variants present
        if 'date' not in df.columns and 'datetime' in df.columns:
            df = df.rename(columns={'datetime': 'date'})
        if 'close' not in df.columns:
            for alt in ['adj_close', 'adjclose', 'closing_price', 'close_price']:
                if alt in df.columns:
                    df = df.rename(columns={alt: 'close'})
                    break
        if 'volume' not in df.columns and 'vol' in df.columns:
            df = df.rename(columns={'vol': 'volume'})
        if 'date' in df.columns:
            df['date'] = pd.to_datetime(df['date'], errors='coerce')
        keep = [c for c in ['date', 'open', 'high', 'low', 'close', 'volume', 'symbol'] if c in df.columns]
        return df[keep] if keep else df

    def _symbol_from_filename(fname: str) -> str:
        base = os.path.basename(fname)
        if base.endswith('_historical.csv'):
            base = base[:-len('_historical.csv')]
        if base.endswith('_NS'):
            return base.replace('_NS', '.NS')
        return base

    have_combined = ('combined_stock_data' in globals() or 'combined_stock_data' in locals())
    if have_combined:
        try:
            _tmp_df = combined_stock_data
            if isinstance(_tmp_df, pd.DataFrame) and not _tmp_df.empty:
                print("Using existing combined_stock_data from previous cells")
            else:
                have_combined = False
        except Exception:
            have_combined = False

    if not have_combined:
        print("combined_stock_data is not available; attempting fallback load from ../data/raw ...")
        project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
        raw_data_path = os.path.join(project_root, 'data', 'raw')
        combined_list = []
        if os.path.exists(raw_data_path):
            csv_files = [f for f in os.listdir(raw_data_path) if f.endswith('_historical.csv')]
            if csv_files:
                print(f"Found {len(csv_files)} *_historical.csv files:")
                for f in csv_files:
                    print(f" - {f}")
                for f in csv_files:
                    fpath = os.path.join(raw_data_path, f)
                    try:
                        df = pd.read_csv(fpath)
                        if df is None or df.empty:
                            continue
                        df = _normalize_df(df)
                        if 'symbol' not in df.columns or df['symbol'].isna().all():
                            df['symbol'] = _symbol_from_filename(f)
                        essential_ok = all(col in df.columns for col in ['date', 'close', 'volume', 'symbol'])
                        if not essential_ok:
                            if 'volume' not in df.columns:
                                df['volume'] = np.nan
                            if 'close' not in df.columns and 'adj_close' in df.columns:
                                df['close'] = df['adj_close']
                            essential_ok = all(col in df.columns for col in ['date', 'close', 'volume', 'symbol'])
                        if essential_ok:
                            combined_list.append(df[['date', 'close', 'volume', 'symbol']].copy())
                    except Exception as e:
                        print(f"Error reading {fpath}: {e}")
            else:
                print("No *_historical.csv files found in data/raw")
        else:
            print(f"data/raw directory not found at: {raw_data_path}")

        if combined_list:
            combined_stock_data = pd.concat(combined_list, ignore_index=True)
            if 'date' in combined_stock_data.columns:
                combined_stock_data = combined_stock_data.dropna(subset=['date'])
            combined_stock_data = combined_stock_data.sort_values(['symbol', 'date'])
            print(f"Successfully loaded {combined_stock_data.shape[0]} rows across {combined_stock_data['symbol'].nunique()} symbols")
        else:
            combined_stock_data = pd.DataFrame()
            print("Fallback load did not produce any data")

    if isinstance(combined_stock_data, pd.DataFrame) and not combined_stock_data.empty:
        for required in ['date', 'close', 'volume', 'symbol']:
            if required not in combined_stock_data.columns:
                raise ValueError(f"Missing required column: {required}")

        print(f"\nCreating visualization for {combined_stock_data['symbol'].nunique()} stocks ...")

        fig = make_subplots(
            rows=2, cols=1,
            shared_xaxes=True,
            vertical_spacing=0.1,
            subplot_titles=('Stock Prices', 'Trading Volume'),
            row_width=[0.3, 0.7]
        )

        colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd']
        symbols = list(combined_stock_data['symbol'].dropna().unique())

        for i, symbol in enumerate(symbols):
            stock_subset = combined_stock_data[combined_stock_data['symbol'] == symbol]
            if stock_subset.empty:
                continue
            color = colors[i % len(colors)]

            fig.add_trace(
                go.Scatter(
                    x=stock_subset['date'],
                    y=stock_subset['close'],
                    mode='lines',
                    name=symbol.replace('.NS', ''),
                    line=dict(color=color, width=2)
                ),
                row=1, col=1
            )

            fig.add_trace(
                go.Bar(
                    x=stock_subset['date'],
                    y=stock_subset['volume'],
                    name=f"{symbol.replace('.NS', '')} Volume",
                    marker_color=color,
                    opacity=0.7,
                    showlegend=False
                ),
                row=2, col=1
            )

        fig.update_layout(
            title='NSE Stock Prices and Volume',
            height=600,
            hovermode='x unified',
            legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1)
        )
        fig.update_yaxes(title_text='Price (₹)', row=1, col=1)
        fig.update_yaxes(title_text='Volume', row=2, col=1)
        fig.update_xaxes(title_text='Date', row=2, col=1)

        fig.show()
        print("Visualization completed successfully!")
    else:
        print("\nNo stock data available for visualization.")
        print("Run the imports and data collection cells first, then re-run this cell.")

except Exception as e:
    print(f"Unexpected error: {e}")
    import traceback
    traceback.print_exc()

Using existing combined_stock_data from previous cells

Creating visualization for 5 stocks ...


Visualization completed successfully!


## 3. Data Preprocessing and Feature Engineering

Now let's preprocess the stock data and create technical indicators for machine learning.

In [50]:
# Preprocess stock data and create technical indicators
try:
    if 'combined_stock_data' in locals() and not combined_stock_data.empty:
        print("Processing stock data...")
        
        # Initialize stock preprocessor
        stock_preprocessor = StockDataPreprocessor()
        
        # Step 1: Clean the data
        cleaned_stock_data = stock_preprocessor.clean_stock_data(combined_stock_data)
        print(f"Cleaned data shape: {cleaned_stock_data.shape}")
        
        # Step 2: Create technical indicators
        stock_with_indicators = stock_preprocessor.create_technical_indicators(cleaned_stock_data)
        print(f"Added technical indicators. New shape: {stock_with_indicators.shape}")
        
        # Step 3: Create target variable for prediction
        stock_with_target = stock_preprocessor.create_target_variable(stock_with_indicators)
        print(f"Created target variable. Final shape: {stock_with_target.shape}")
        
        # Show target distribution
        print("\n Target Distribution (Next Day Movement):")
        target_counts = stock_with_target['target'].value_counts()
        print(target_counts)
        
        # Show some technical indicators
        tech_indicators = [col for col in stock_with_indicators.columns if col in [
            'sma_5', 'sma_20', 'ema_12', 'ema_26', 'rsi', 'macd', 'daily_return', 'volatility'
        ]]
        
        print(f"\n Technical indicators created: {tech_indicators}")
        
        # Display sample of processed data
        print("\n Sample of processed data with indicators:")
        display(stock_with_indicators[['date', 'symbol', 'close', 'daily_return', 'sma_5', 'sma_20', 'rsi']].head(10))
        
    else:
        print(" No stock data to process")
        print(" Please run the data collection cell first")
        import pandas as pd
        stock_with_target = pd.DataFrame()
        
except NameError as e:
    print(f"Import error: {e}")
    print("Please run cell 2 first to import all required modules")
    import pandas as pd
    stock_with_target = pd.DataFrame()
    
except Exception as e:
    print(f" Error processing stock data: {e}")
    import pandas as pd
    stock_with_target = pd.DataFrame()

INFO:src.preprocessing:Cleaning stock data...
INFO:src.preprocessing:Cleaned data shape: (2475, 7)
INFO:src.preprocessing:Creating technical indicators...
INFO:src.preprocessing:Cleaned data shape: (2475, 7)
INFO:src.preprocessing:Creating technical indicators...
INFO:src.preprocessing:Created technical indicators. New shape: (2475, 54)
INFO:src.preprocessing:Creating target variable...
INFO:src.preprocessing:Target distribution:
target
DOWN       1221
UP         1189
NEUTRAL      65
Name: count, dtype: int64
INFO:src.preprocessing:Target distribution:
target
DOWN       1221
UP         1189
NEUTRAL      64
Name: count, dtype: int64
INFO:src.preprocessing:Created technical indicators. New shape: (2475, 54)
INFO:src.preprocessing:Creating target variable...
INFO:src.preprocessing:Target distribution:
target
DOWN       1221
UP         1189
NEUTRAL      65
Name: count, dtype: int64
INFO:src.preprocessing:Target distribution:
target
DOWN       1221
UP         1189
NEUTRAL      64
Name: coun

Processing stock data...
Cleaned data shape: (2475, 7)
Added technical indicators. New shape: (2475, 54)
Created target variable. Final shape: (2474, 62)

 Target Distribution (Next Day Movement):
target
DOWN       1221
UP         1189
NEUTRAL      64
Name: count, dtype: int64

 Technical indicators created: ['daily_return', 'sma_5', 'sma_20', 'ema_12', 'ema_26', 'macd', 'rsi', 'volatility']

 Sample of processed data with indicators:


,date,symbol,close,daily_return,sma_5,sma_20,rsi
1485,2023-11-13,HDFCBANK.NS,724.577759,NaN,NaN,NaN,NaN
1980,2023-11-13,ICICIBANK.NS,919.380188,0.268850,NaN,NaN,NaN
495,2023-11-13,TCS.NS,3159.151855,2.436176,NaN,NaN,NaN
0,2023-11-13,RELIANCE.NS,1148.797241,-0.636359,NaN,NaN,NaN
990,2023-11-13,INFY.NS,1293.176636,0.125679,1449.016736,NaN,NaN
1486,2023-11-15,HDFCBANK.NS,732.170044,-0.433821,1450.535193,NaN,NaN
496,2023-11-15,TCS.NS,3228.137939,3.409000,1912.286743,NaN,NaN
1981,2023-11-15,ICICIBANK.NS,925.922913,-0.713171,1465.640955,NaN,NaN
1,2023-11-15,RELIANCE.NS,1169.568481,0.263138,1469.795203,NaN,NaN
991,2023-11-15,INFY.NS,1327.964355,0.135431,1476.752747,NaN,NaN


## 4. Sentiment Analysis

If news data is available, let's analyze the sentiment of financial news.

In [51]:
# Perform sentiment analysis on news data
try:
    if 'news_data' in locals() and not news_data.empty:
        print("Analyzing news sentiment...")
        
        # Check if SentimentAnalyzer is available
        if 'SentimentAnalyzer' in globals():
            # Initialize sentiment analyzer
            sentiment_analyzer = SentimentAnalyzer(['vader', 'textblob'])
            
            # Perform sentiment analysis
            news_with_sentiment = sentiment_analyzer.analyze_dataframe(news_data)
            print(f"Added sentiment features. Shape: {news_with_sentiment.shape}")
            
            # Show sentiment distribution
            if 'vader_label' in news_with_sentiment.columns:
                print("\n VADER Sentiment Distribution:")
                vader_counts = news_with_sentiment['vader_label'].value_counts()
                print(vader_counts)
                
                # Create sentiment pie chart
                fig = px.pie(
                    values=vader_counts.values, 
                    names=vader_counts.index,
                    title="News Sentiment Distribution (VADER)",
                    color_discrete_sequence=px.colors.qualitative.Set3
                )
                fig.show()
            
            # Show sentiment over time
            if 'date' in news_with_sentiment.columns and 'vader_compound' in news_with_sentiment.columns:
                daily_sentiment = news_with_sentiment.groupby('date')['vader_compound'].mean().reset_index()
                
                fig = px.line(
                    daily_sentiment, 
                    x='date', 
                    y='vader_compound',
                    title='Average Daily News Sentiment Score',
                    labels={'vader_compound': 'Sentiment Score', 'date': 'Date'}
                )
                fig.add_hline(y=0, line_dash="dash", line_color="red", annotation_text="Neutral")
                fig.show()
            
            # Sample sentiment analysis results
            print("\n📰 Sample News with Sentiment:")
            sample_cols = ['title', 'vader_label', 'vader_compound', 'textblob_label', 'textblob_polarity']
            available_cols = [col for col in sample_cols if col in news_with_sentiment.columns]
            display(news_with_sentiment[available_cols].head())
            
        else:
            print("SentimentAnalyzer not available")
            print(" Please run cell 2 first to import SentimentAnalyzer")
            print(" Creating basic sentiment analysis fallback...")
            
            # Basic sentiment analysis fallback using VADER only
            try:
                from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
                import pandas as pd
                
                analyzer = SentimentIntensityAnalyzer()
                news_with_sentiment = news_data.copy()
                
                # Apply VADER sentiment to each news item
                sentiments = []
                for text in news_data['title'].fillna(''):
                    if text:
                        scores = analyzer.polarity_scores(text)
                        sentiments.append({
                            'vader_compound': scores['compound'],
                            'vader_pos': scores['pos'],
                            'vader_neu': scores['neu'], 
                            'vader_neg': scores['neg']
                        })
                    else:
                        sentiments.append({
                            'vader_compound': 0.0,
                            'vader_pos': 0.0,
                            'vader_neu': 1.0,
                            'vader_neg': 0.0
                        })
                
                # Add sentiment scores to dataframe
                sentiment_df = pd.DataFrame(sentiments)
                news_with_sentiment = pd.concat([news_with_sentiment, sentiment_df], axis=1)
                
                # Add sentiment labels
                news_with_sentiment['vader_label'] = news_with_sentiment['vader_compound'].apply(
                    lambda x: 'positive' if x >= 0.05 else 'negative' if x <= -0.05 else 'neutral'
                )
                
                print(f" Basic sentiment analysis completed. Shape: {news_with_sentiment.shape}")
                print(f" Sentiment distribution: {news_with_sentiment['vader_label'].value_counts().to_dict()}")
                
                # Show sample results
                print("\n Sample News with Basic Sentiment:")
                display(news_with_sentiment[['title', 'vader_label', 'vader_compound']].head())
                
            except ImportError:
                print(" VADER sentiment library not available")
                print(" Install with: pip install vaderSentiment")
                import pandas as pd
                news_with_sentiment = pd.DataFrame()
        
    else:
        print(" No news data available for sentiment analysis")
        print(" To enable news sentiment analysis:")
        print(" 1. Make sure the news collection cell ran successfully")
        print(" 2. Check API keys in .env file")
        print(" 3. Re-run the news collection cell")
        
        # Create empty sentiment data for demonstration
        import pandas as pd
        news_with_sentiment = pd.DataFrame()
        
except NameError as e:
    print(f" Import error: {e}")
    print(" Please run cell 2 first to import all required modules")
    import pandas as pd
    news_with_sentiment = pd.DataFrame()
    
except Exception as e:
    print(f"Error in sentiment analysis: {e}")
    import pandas as pd
    news_with_sentiment = pd.DataFrame()

INFO:src.sentiment:Initialized sentiment analyzers: ['vader', 'textblob']
INFO:src.sentiment:Analyzing sentiment for 360 texts...
INFO:src.sentiment:Processed 0/360 texts
INFO:src.sentiment:Analyzing sentiment for 360 texts...
INFO:src.sentiment:Processed 0/360 texts


Analyzing news sentiment...


INFO:src.sentiment:Processed 100/360 texts
INFO:src.sentiment:Processed 200/360 texts
INFO:src.sentiment:Processed 200/360 texts
INFO:src.sentiment:Processed 300/360 texts
INFO:src.sentiment:Processed 300/360 texts
INFO:src.sentiment:Completed sentiment analysis. Added 20 sentiment features.
INFO:src.sentiment:Completed sentiment analysis. Added 20 sentiment features.


Added sentiment features. Shape: (360, 30)

 VADER Sentiment Distribution:
vader_label
POSITIVE    248
NEGATIVE     73
NEUTRAL      39
Name: count, dtype: int64



📰 Sample News with Sentiment:


,title,vader_label,vader_compound,textblob_label,textblob_polarity
0,"RIL stock jumps 10% in 1 month, nears record h...",POSITIVE,0.2960,POSITIVE,0.160000
1,"Goldman Sachs says Reliance Industries, Titan ...",POSITIVE,0.0516,NEUTRAL,-0.062500
2,"Stock market today: Trade setup for Nifty 50, ...",POSITIVE,0.7579,POSITIVE,0.166667
3,FIIs trim stake in Reliance Industries in Q2 b...,POSITIVE,0.8844,POSITIVE,0.187500
4,Reliance Industries shares tumble: Why the con...,POSITIVE,0.5986,NEGATIVE,-0.125000


## 5. Model Training and Evaluation

Now let's train machine learning models to predict stock movements.

In [52]:
# Prepare data for machine learning
if not stock_with_target.empty and len(stock_with_target) > 50:
    print("Preparing data for machine learning...")
    
    # Integrate stock and news data if both available
    if not news_with_sentiment.empty:
        print("Integrating stock and news data...")
        integrator = DataIntegrator()
        try:
            ml_data = integrator.align_data_by_date(stock_with_target, news_with_sentiment)
            print(f" Integrated data shape: {ml_data.shape}")
        except KeyError as e:
            print(f"Error during integration: {e}")
            print("Ensure all required columns are present in news_with_sentiment.")
            ml_data = stock_with_target.copy()
    else:
        print(" Using stock data only...")
        ml_data = stock_with_target.copy()
    
    # Prepare features for ML
    stock_preprocessor = StockDataPreprocessor()
    try:
        ml_ready_data = stock_preprocessor.prepare_features(ml_data)
        
        # Remove rows with missing targets
        ml_ready_data = ml_ready_data.dropna(subset=['target'])
        
        print(f" ML-ready data shape: {ml_ready_data.shape}")
        print(f" Final target distribution:")
        print(ml_ready_data['target'].value_counts())
    except Exception as e:
        print(f"Error during feature preparation: {e}")
        ml_ready_data = pd.DataFrame()
    
    # Check if all classes have at least 2 samples
    if not ml_ready_data.empty:
        target_counts = ml_ready_data['target'].value_counts()
        if (target_counts < 2).any():
            print(" Not enough samples in one or more classes. Each class must have at least 2 samples.")
            print("Consider collecting more data or adjusting the target variable.")
        elif len(ml_ready_data) >= 50:
            # Train a simple logistic regression model for demonstration
            print("\n Training Logistic Regression model...")
            
            lr_model = LogisticRegressionModel()
            try:
                metrics = lr_model.train(ml_ready_data, target_col='target')
                
                print("\n Model Performance:")
                for metric, value in metrics.items():
                    print(f"  {metric.capitalize()}: {value:.4f}")
                
                # Get feature importance
                if lr_model.is_fitted:
                    feature_importance = lr_model.get_feature_importance()
                    
                    print("\n🔝 Top 10 Most Important Features:")
                    display(feature_importance.head(10))
                    
                    # Plot feature importance
                    top_features = feature_importance.head(15)
                    
                    fig = px.bar(
                        top_features,
                        x='importance',
                        y='feature',
                        orientation='h',
                        title='Top 15 Most Important Features',
                        labels={'importance': 'Feature Importance', 'feature': 'Feature'}
                    )
                    fig.update_layout(height=500)
                    fig.show()
            except Exception as e:
                print(f"Error during model training: {e}")
        else:
            print(f"Not enough data for training. Need at least 50 samples, got {len(ml_ready_data)}")
    else:
        print(" No processed stock data available for model training")
else:
    print(" No processed stock data available for model training")
    print(" Make sure stock data collection was successful")

INFO:src.preprocessing:Aligning stock and news data by date...
INFO:src.preprocessing:Preparing features for ML...
INFO:src.preprocessing:Preparing features for ML...
INFO:src.preprocessing:Prepared 52 features for ML
INFO:src.model:Training Logistic_Regression model...
INFO:src.model:Preparing data for Logistic_Regression...
INFO:src.preprocessing:Prepared 52 features for ML
INFO:src.model:Training Logistic_Regression model...
INFO:src.model:Preparing data for Logistic_Regression...


Preparing data for machine learning...
Integrating stock and news data...
Error during integration: "Column(s) ['content_length', 'content_word_count', 'description_length', 'description_word_count', 'title_length', 'title_word_count', 'total_financial_keywords'] do not exist"
Ensure all required columns are present in news_with_sentiment.
 ML-ready data shape: (2474, 62)
 Final target distribution:
target
DOWN       1221
UP         1189
NEUTRAL      64
Name: count, dtype: int64

 Training Logistic Regression model...


INFO:src.model:Data prepared: Train shape (1979, 52), Test shape (495, 52)
INFO:src.model:Logistic_Regression Performance:
INFO:src.model:  accuracy: 0.7677
INFO:src.model:  precision: 0.7483
INFO:src.model:  recall: 0.7677
INFO:src.model:  f1_score: 0.7575
INFO:src.model:Logistic_Regression Performance:
INFO:src.model:  accuracy: 0.7677
INFO:src.model:  precision: 0.7483
INFO:src.model:  recall: 0.7677
INFO:src.model:  f1_score: 0.7575



 Model Performance:
  Accuracy: 0.7677
  Precision: 0.7483
  Recall: 0.7677
  F1_score: 0.7575

🔝 Top 10 Most Important Features:


,feature,importance
31,bb_position,0.795363
38,support_distance,0.725144
36,support,0.638732
39,resistance_distance,0.621861
26,rsi,0.442395
37,resistance,0.426345
4,volume,0.419868
25,macd_diff,0.409987
34,price_volume,0.393034
5,daily_return,0.352324


## 6. Technical Analysis Visualizations

Let's create some technical analysis charts to understand stock patterns better.

In [53]:
# Create technical analysis charts
if 'stock_with_indicators' in locals() and not stock_with_indicators.empty:
    # Select one stock for detailed technical analysis
    selected_symbol = stock_with_indicators['symbol'].iloc[0]
    symbol_data = stock_with_indicators[stock_with_indicators['symbol'] == selected_symbol].copy()
    symbol_data = symbol_data.sort_values('date')
    
    print(f" Creating technical analysis charts for {selected_symbol}")
    
    # Create comprehensive technical analysis chart
    fig = make_subplots(
        rows=4, cols=1,
        shared_xaxes=True,
        vertical_spacing=0.05,
        subplot_titles=(
            f'{selected_symbol.replace(".NS", "")} - Price with Moving Averages',
            'RSI (Relative Strength Index)',
            'MACD',
            'Volume'
        ),
        row_heights=[0.4, 0.2, 0.2, 0.2]
    )
    
    # Row 1: Price with SMA and EMA
    fig.add_trace(go.Scatter(
        x=symbol_data['date'], y=symbol_data['close'],
        mode='lines', name='Close Price', line=dict(color='blue', width=2)
    ), row=1, col=1)
    
    if 'sma_20' in symbol_data.columns:
        fig.add_trace(go.Scatter(
            x=symbol_data['date'], y=symbol_data['sma_20'],
            mode='lines', name='SMA 20', line=dict(color='orange', width=1)
        ), row=1, col=1)
    
    if 'ema_12' in symbol_data.columns:
        fig.add_trace(go.Scatter(
            x=symbol_data['date'], y=symbol_data['ema_12'],
            mode='lines', name='EMA 12', line=dict(color='green', width=1)
        ), row=1, col=1)
    
    # Row 2: RSI
    if 'rsi' in symbol_data.columns:
        fig.add_trace(go.Scatter(
            x=symbol_data['date'], y=symbol_data['rsi'],
            mode='lines', name='RSI', line=dict(color='purple')
        ), row=2, col=1)
        
        # Add RSI levels
        fig.add_hline(y=70, line_dash="dash", line_color="red", row=2, col=1)
        fig.add_hline(y=30, line_dash="dash", line_color="green", row=2, col=1)
    
    # Row 3: MACD
    if 'macd' in symbol_data.columns:
        fig.add_trace(go.Scatter(
            x=symbol_data['date'], y=symbol_data['macd'],
            mode='lines', name='MACD', line=dict(color='blue')
        ), row=3, col=1)
        
        if 'macd_signal' in symbol_data.columns:
            fig.add_trace(go.Scatter(
                x=symbol_data['date'], y=symbol_data['macd_signal'],
                mode='lines', name='MACD Signal', line=dict(color='red')
            ), row=3, col=1)
    
    # Row 4: Volume
    fig.add_trace(go.Bar(
        x=symbol_data['date'], y=symbol_data['volume'],
        name='Volume', marker_color='lightblue'
    ), row=4, col=1)
    
    # Update layout
    fig.update_layout(
        title=f'Technical Analysis - {selected_symbol.replace(".NS", "")}',
        height=800,
        showlegend=True,
        hovermode='x unified'
    )
    
    # Update y-axis labels
    fig.update_yaxes(title_text="Price (₹)", row=1, col=1)
    fig.update_yaxes(title_text="RSI", row=2, col=1, range=[0, 100])
    fig.update_yaxes(title_text="MACD", row=3, col=1)
    fig.update_yaxes(title_text="Volume", row=4, col=1)
    
    fig.show()
    
    # Daily Returns Distribution
    if 'daily_return' in symbol_data.columns:
        fig_returns = px.histogram(
            symbol_data,
            x='daily_return',
            nbins=50,
            title=f'Daily Returns Distribution - {selected_symbol.replace(".NS", "")}',
            labels={'daily_return': 'Daily Return', 'count': 'Frequency'}
        )
        fig_returns.add_vline(x=0, line_dash="dash", line_color="red", annotation_text="Zero Return")
        fig_returns.show()
        
        # Calculate some statistics
        returns = symbol_data['daily_return'].dropna()
        print(f"\\n Daily Returns Statistics for {selected_symbol}:")
        print(f"  Mean: {returns.mean():.4f}")
        print(f"  Std Dev: {returns.std():.4f}")
        print(f"  Skewness: {returns.skew():.4f}")
        print(f"  Min: {returns.min():.4f}")
        print(f"  Max: {returns.max():.4f}")
else:
    print(" No technical indicators data available for visualization")

 Creating technical analysis charts for HDFCBANK.NS


\n Daily Returns Statistics for HDFCBANK.NS:
  Mean: -0.4526
  Std Dev: 0.2036
  Skewness: -0.1984
  Min: -0.8270
  Max: 0.0463


## 7. Next Steps and Conclusions

This notebook demonstrated the complete workflow of the NEWS2PROFIT project:

### What We've Accomplished:
1. **Data Collection**: Fetched stock data from yfinance and news data from multiple sources
2.  **Data Preprocessing**: Cleaned data and created technical indicators (SMA, EMA, RSI, MACD)
3.  **Feature Engineering**: Generated predictive features and target variables
4. **Sentiment Analysis**: Analyzed news sentiment using VADER and TextBlob
5.  **Model Training**: Trained machine learning models for stock movement prediction
6. **Visualization**: Created interactive charts for analysis

### Key Findings:
- Technical indicators provide valuable signals for stock movement prediction
- News sentiment can add additional predictive power when available
- Multiple models can be compared to find the best performing approach

### Next Steps:
1. **Add More Data Sources**: Include more financial news APIs and social media data
2. **Advanced Models**: Implement XGBoost and LSTM models for better performance  
3. **Real-time Predictions**: Set up automated daily predictions
4. **Risk Management**: Add portfolio optimization and risk metrics
5. **Dashboard Deployment**: Launch the Streamlit dashboard for live monitoring

### How to Run the Complete Pipeline:
```bash
# 1. Set up environment variables
cp .env.template .env
# Edit .env with your API keys

# 2. Run the full training pipeline
python train_models.py

# 3. Launch the interactive dashboard
streamlit run src/dashboard.py

# 4. Run tests
python -m pytest tests/
```

### Resources:
-  [Project Documentation](../README.md)
-  [Configuration Settings](../config/config.py)
-  [Unit Tests](../tests/test_modules.py)
-  [Dashboard](../src/dashboard.py)